In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from timm.models.vision_transformer import vit_base_patch16_224
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
%matplotlib inline

In [3]:
train_df = pd.read_csv('/kaggle/input/isic-2017/ISIC-2017_Training_Part3_GroundTruth.csv')
validation_df = pd.read_csv('/kaggle/input/isic-2017/ISIC-2017_Validation_Part3_GroundTruth.csv')
print(len(train_df), len(validation_df))

2000 150


In [4]:
# setup the dataloader
class ISICDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        # self.data['melanoma'] = self.data['melanoma'].astype('float64')
        # self.data['seborrheic_keratosis'] = self.data['seborrheic_keratosis'].astype('float64')
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_id = self.data.loc[idx, 'image_id']
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        melanoma = self.data.loc[idx, 'melanoma']
        seborrheic_keratosis = self.data.loc[idx, 'seborrheic_keratosis']
        if self.transform:
            image = self.transform(image)
        return image, melanoma, seborrheic_keratosis
    
# path of dataset
train_csv_file = "/kaggle/input/isic-2017/ISIC-2017_Training_Part3_GroundTruth.csv"
train_img_dir = "/kaggle/input/isic-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data"  # image directory
validation_csv_file = "/kaggle/input/isic-2017/ISIC-2017_Validation_Part3_GroundTruth.csv"
validation_img_dir = "/kaggle/input/isic-2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data"  # image directory

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# create dataloader
train_dataset = ISICDataset(train_csv_file, train_img_dir, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataset = ISICDataset(validation_csv_file, validation_img_dir, transform=transform)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

In [6]:
print(len(train_dataloader))
# test the dataloader
# get the first batch
for image, melanoma, seborrheic_keratosis in train_dataloader:
    print("Image shape:", image.shape)
    print("Melanoma label:", melanoma)
    print("Seborrheic Keratosis label:", seborrheic_keratosis)
    break

63
Image shape: torch.Size([32, 3, 224, 224])
Melanoma label: tensor([0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
Seborrheic Keratosis label: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       dtype=torch.float64)


**1 Classification task for (a) melanoma and (b) nevus and seborrheic keratosis**


In [7]:
from timm import create_model

torch.cuda.empty_cache()
# load the pretrained model and modify the last layer
model = create_model('vit_base_patch16_224', pretrained=True)

num_features = model.head.in_features
# print(f"num features: {num_features}")  # 768
model.head = nn.Linear(num_features, 2)  # binary classification

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

epochs = 10
device = "cuda"  # "cuda"
model = model.to(device)
model.train()
for epoch in range(epochs):
    i = 0
    for images, labels, _ in train_dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        outputs = model(images)
        labels = labels.long().to(device)
        # print(type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        i += 1
        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, i: {i} Loss: {loss.item()}")
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1/10, i: 10 Loss: 0.5316329598426819
Epoch 1/10, i: 20 Loss: 0.5777422785758972
Epoch 1/10, i: 30 Loss: 0.44979873299598694
Epoch 1/10, i: 40 Loss: 0.21224676072597504
Epoch 1/10, i: 50 Loss: 0.33057519793510437
Epoch 1/10, i: 60 Loss: 0.3104608952999115
Epoch 1/10, Loss: 0.35190531611442566
Epoch 2/10, i: 10 Loss: 0.34770849347114563
Epoch 2/10, i: 20 Loss: 0.39372551441192627
Epoch 2/10, i: 30 Loss: 0.34599635004997253
Epoch 2/10, i: 40 Loss: 0.17201152443885803
Epoch 2/10, i: 50 Loss: 0.33376234769821167
Epoch 2/10, i: 60 Loss: 0.7705468535423279
Epoch 2/10, Loss: 0.1775539070367813
Epoch 3/10, i: 10 Loss: 0.3845512568950653
Epoch 3/10, i: 20 Loss: 0.1855851262807846
Epoch 3/10, i: 30 Loss: 0.23701798915863037
Epoch 3/10, i: 40 Loss: 0.25255995988845825
Epoch 3/10, i: 50 Loss: 0.1798211932182312
Epoch 3/10, i: 60 Loss: 0.1317732185125351
Epoch 3/10, Loss: 0.26538243889808655
Epoch 4/10, i: 10 Loss: 0.13510410487651825
Epoch 4/10, i: 20 Loss: 0.06410743296146393
Epoch 4/10, i: 

In [8]:
# save model
model_path = "/kaggle/working/model_task1.pth"
torch.save(model.state_dict(), model_path)

In [9]:
# reload model
model_path = "/kaggle/working/model_task1.pth"

# create empty model with same structure
model = create_model('vit_base_patch16_224', pretrained=False)
num_features = model.head.in_features
model.head = nn.Linear(num_features, 2)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [11]:
# validation
from sklearn.metrics import accuracy_score

device = "cuda:0"
model = model.to(device)
model.eval()

# 定义验证指标
valid_loss = 0.0
valid_preds = []
valid_labels = []
criterion = nn.CrossEntropyLoss()

# validation
with torch.no_grad():
    for images, labels, _ in validation_dataloader:
        images = images.to(device)
        outputs = model(images)
        labels = labels.long().to(device)
        # print(type(labels), type(outputs))
        loss = criterion(outputs, labels)
        valid_loss += loss.item() * images.size(0)  # accumulated loss

        preds = torch.argmax(outputs, dim=1)
        valid_preds.extend(preds.tolist())  # predictions
        valid_labels.extend(labels.tolist())  # gt labels

valid_loss /= len(validation_dataloader) * 32  # average loss
valid_accuracy = accuracy_score(valid_labels, valid_preds)  # accuracy

print("Validation Loss: {:.4f}".format(valid_loss))
print("Validation Accuracy: {:.4f}".format(valid_accuracy))

Validation Loss: 0.6268
Validation Accuracy: 0.8867


**2 Classification task for (a) seborrheic keratosis and (b) nevus and melanoma**

In [13]:
from tqdm import tqdm
from timm import create_model

torch.cuda.empty_cache()
# load the pretrained model and modify the last layer
model = create_model('vit_base_patch16_224', pretrained=True)
num_features = model.head.in_features
# print(f"num features: {num_features}")  # 768
model.head = nn.Linear(num_features, 2)  # binary classification

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

epochs = 10
device = "cpu"  # "cuda"
model = model.to(device)
model.train()
for epoch in range(epochs):
    i = 0
    for images, _, labels in train_dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        outputs = model(images)
        labels = labels.long().to(device)
        # print(type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        i += 1
        if i % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, i: {i} Loss: {loss.item()}")
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, i: 10 Loss: 0.13759838044643402
Epoch 1/10, i: 20 Loss: 0.3535006642341614
Epoch 1/10, i: 30 Loss: 0.28682219982147217
Epoch 1/10, i: 40 Loss: 0.4464345872402191
Epoch 1/10, i: 50 Loss: 0.15934856235980988
Epoch 1/10, i: 60 Loss: 0.3907676339149475
Epoch 1/10, Loss: 0.2381468266248703
Epoch 2/10, i: 10 Loss: 0.13791921734809875
Epoch 2/10, i: 20 Loss: 0.1263595074415207
Epoch 2/10, i: 30 Loss: 0.04496702551841736
Epoch 2/10, i: 40 Loss: 0.1255866438150406
Epoch 2/10, i: 50 Loss: 0.10331703722476959
Epoch 2/10, i: 60 Loss: 0.031362153589725494
Epoch 2/10, Loss: 0.46935006976127625
Epoch 3/10, i: 10 Loss: 0.08370763063430786
Epoch 3/10, i: 20 Loss: 0.016115229576826096
Epoch 3/10, i: 30 Loss: 0.03680095449090004
Epoch 3/10, i: 40 Loss: 0.024526802822947502
Epoch 3/10, i: 50 Loss: 0.04333113133907318
Epoch 3/10, i: 60 Loss: 0.029896272346377373
Epoch 3/10, Loss: 0.04091682657599449
Epoch 4/10, i: 10 Loss: 0.019281703978776932
Epoch 4/10, i: 20 Loss: 0.027721894904971123
Epoch 

In [14]:
# save model
model_path = "/kaggle/working/model_task2.pth"
torch.save(model.state_dict(), model_path)

In [15]:
# reload model
model_path = "/kaggle/working/model_task2.pth"

# create empty model with same structure
model = create_model('vit_base_patch16_224', pretrained=False)
num_features = model.head.in_features
model.head = nn.Linear(num_features, 2)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [17]:
from sklearn.metrics import accuracy_score

device = "cuda"  # "cuda"
model = model.to(device)
model.eval()

# 定义验证指标
valid_loss = 0.0
valid_preds = []
valid_labels = []
criterion = nn.CrossEntropyLoss()

# validation
with torch.no_grad():
    for images, _, labels in validation_dataloader:
        images = images.to(device)
        outputs = model(images)
        labels = labels.long().to(device)
        # print(type(labels), type(outputs))
        loss = criterion(outputs, labels)
        valid_loss += loss.item() * images.size(0)  # accumulated loss

        preds = torch.argmax(outputs, dim=1)
        valid_preds.extend(preds.tolist())  # predictions
        valid_labels.extend(labels.tolist())  # gt labels

valid_loss /= len(validation_dataloader)*32  # average loss
valid_accuracy = accuracy_score(valid_labels, valid_preds)  # accuracy

print("Validation Loss: {:.4f}".format(valid_loss))
print("Validation Accuracy: {:.4f}".format(valid_accuracy))

Validation Loss: 0.2940
Validation Accuracy: 0.9267
